In [26]:
import os
import pandas as pd
import plotly.express as px

In [27]:
indicators = [file for file in os.listdir('data/indicator') if len(file) == 3]

In [28]:
dfs = []

for indicator in indicators:
    for status in ['processed', 'preprocessed']:
        path = f'data/indicator/{indicator}/{status}'
        files = os.listdir(path)
        for file in files:
            df = pd.read_csv(f'{path}/{file}')
            df['status'] = status
            
            if status == 'processed':
                df['Variable'] = df['Indicator']
                
            df['Indicator'] = df['Variable'].apply(lambda x: x.split('.')[0])
                
            dfs.append(df)

In [29]:
data = pd.concat(dfs, axis=0)

In [34]:
indicator = 'SE1'
ISO = ['FRA']
df = data[(data.Indicator == indicator) & (data.ISO.isin(ISO))]

fig =px.scatter(df,
           x='Year',
           y='Value',
           facet_col='Variable',
           facet_col_wrap=2,
           symbol='From',
           color='status',
           hover_data={'Source': False,
                       'Description': False,
                       'Corrected': True,
                       'Imputed': True})
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=True, col=2)

In [31]:
from hampel import hampel


def filter_outliers_from_ISO(df):
    ts = hampel(df['Value'].reset_index(drop=True), window_size=3, n=1).values
    df.loc[:, 'filtered_Value'] = ts
    df['Corrected'] = abs(df['filtered_Value'] - df['Value']) > 1e-3
    return df


In [ ]:
variable = 'SE1.1'
